The Copernicus Data Space Ecosystem (CDSE) is a comprehensive platform that provides access to a wide range of Earth observation data and services. It is designed to facilitate the discovery, access, and utilization of Copernicus data for various applications, including environmental monitoring, climate change analysis, and resource management.

This Jupyter Notebook is specifically tailored for users who are interested in downloading individual files in the .SAFE format from the CDSE. The .SAFE format is a standardized packaging format for Copernicus Sentinel satellite data, which includes both the original imagery and associated metadata.

By using this notebook, users can easily access and download specific .SAFE files from the CDSE, enabling them to work with the data in their preferred software environment. Whether you are a researcher, developer, or data enthusiast, this notebook provides a streamlined workflow for obtaining the desired .SAFE files from the Copernicus Data Space Ecosystem.

In [1]:
import EOVoxelCraft as eovc
import geopandas as gpd
import json

In [2]:
with open("/dss/dsshome1/0F/di35viw/EOVoxelCraft/demo_files/credentials.json") as file:
    credentials = json.load(file)

cdse = eovc.init('cdse', credentials)

In [3]:
collections = cdse.retrieve_collections()
print(collections)

['COP-DEM', 'S2GLC', 'TERRAAQUA', 'SENTINEL-3', 'SENTINEL-5P', 'SENTINEL-1-RTC', 'SENTINEL-1', 'SMOS', 'LANDSAT-7', 'CCM', 'LANDSAT-5', 'LANDSAT-8', 'ENVISAT', 'SENTINEL-6', 'GLOBAL-MOSAICS', 'SENTINEL-2']


In [4]:
arguments = dict(
    shp=gpd.read_file("/dss/dsshome1/0F/di35viw/EOVoxelCraft/demo_files/data/TUM_OTN.shp.zip"), 
    resolution=20,
    collection='SENTINEL-2', 
    start_date='2021-06-01', 
    end_date='2021-07-30', 
    download_folder='/dss/dsshome1/0F/di35viw/EOVoxelCraft/demo_files/downloads', 
    num_workers=4
    )

In [5]:
items = cdse.search(**arguments)

In [10]:
from datetime import datetime
from pystac import ItemCollection

def filter_unique_items(items, preferred_level, tile_id, max_cloud_cover=50):
    def extract(item, key, default=None):
        # Extracts values from properties or defaults if not found
        return item.properties.get(key, default)

    seen = {}
    for item in items:
        if extract(item, 'tileId') != tile_id:
            continue  # Skip items that don't match the specified tileId

        date = datetime.strptime(item.id.split('_')[2][:8], '%Y%m%d').date()
        level, cloud_cover = extract(item, 'productType'), extract(item, 'cloudCover', 100)
        key = (date, tile_id)

        if key not in seen or \
           (level == preferred_level and extract(seen[key], 'productType') != preferred_level) or \
           (level == extract(seen[key], 'productType') and cloud_cover < extract(seen[key], 'cloudCover', 100)):

            if cloud_cover <= max_cloud_cover:
                seen[key] = item

    return ItemCollection(seen.values())

# Example usage:
items_filtered = filter_unique_items(items, preferred_level='S2MSI2A', tile_id='32UPU', max_cloud_cover=50)
items_filtered


Selected items: 6


[<Item id=S2B_MSIL2A_20210710T101559_N0301_R065_T32UQU_20210710T132321.SAFE>,
 <Item id=S2A_MSIL2A_20210722T101031_N0301_R022_T32UQU_20210722T120214.SAFE>,
 <Item id=S2A_MSIL2A_20210722T101031_N0500_R022_T32UQU_20230123T074238.SAFE>,
 <Item id=S2B_MSIL2A_20210617T100559_N0300_R022_T32UQU_20210617T131219.SAFE>,
 <Item id=S2A_MSIL2A_20210712T101031_N0500_R022_T32UQU_20230206T131341.SAFE>,
 <Item id=S2A_MSIL2A_20210712T101031_N0301_R022_T32UQU_20210712T132219.SAFE>]

In [11]:
item_dirs = cdse.download(selected_items, create_minicube=False, delete_zip=True)
#cube = crafter.download(selected_items, create_minicube=True, delete_zip=True)

Failed to download file. Status code: 401
{"detail":"Expired signature!"}
Failed to download file. Status code: 401
{"detail":"Expired signature!"}
Unzipping to temporary directory: /dss/dsstbyfs02/scratch/0F/di35viw/di35viw/tmp0g6jwrif
Extraction to output directory complete: /dss/dsshome1/0F/di35viw/EOVoxelCraft/demo_files/downloads/S2B_MSIL2A_20210710T101559_N0301_R065_T32UQU_20210710T132321.SAFE
Extraction to output directory complete: /dss/dsshome1/0F/di35viw/EOVoxelCraft/demo_files/downloads/S2A_MSIL2A_20210722T101031_N0301_R022_T32UQU_20210722T120214.SAFE
Extraction to output directory complete: /dss/dsshome1/0F/di35viw/EOVoxelCraft/demo_files/downloads/S2A_MSIL2A_20210722T101031_N0500_R022_T32UQU_20230123T074238.SAFE
Extraction to output directory complete: /dss/dsshome1/0F/di35viw/EOVoxelCraft/demo_files/downloads/S2B_MSIL2A_20210617T100559_N0300_R022_T32UQU_20210617T131219.SAFE


In [13]:
ds = crafter.build_minicube(arguments['download_folder'])

In [27]:
ds = build_minicube(output_dir='/dss/dsshome1/0F/di35viw/EOVoxelCraft/demo_files/downloads', shp='/dss/dsshome1/0F/di35viw/EOVoxelCraft/demo_files/data/TUM_OTN.shp.zip', target_res=10, num_workers=4)

In [34]:
ds.rio.bounds()

(696040.0, 5323140.0, 701060.0, 5327460.0)

In [35]:
gdf = gpd.read_file('/dss/dsshome1/0F/di35viw/EOVoxelCraft/demo_files/data/TUM_OTN.shp.zip')

In [39]:
gdf = gdf.to_crs(ds.rio.crs)
gdf.total_bounds

array([ 696044.56842919, 5323147.98448889,  701053.32696745,
       5327459.48676893])